In [ ]:
import os
import warnings
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import anndata as ad
import lightning as L
from os.path import join
from modlyn.io.loading import read_lazy

import lamindb as ln

from modlyn.io.datamodules import ClassificationDataModule
from modlyn.models.linear import Linear
from modlyn.io.loading import read_lazy

In [ ]:
# artifact = ln.Artifact.get("TuhkPw0wkzlUXN5k0000")
# artifact = ln.Artifact.get("BQ6RplqNcT0akokn0000")

store_path = Path("/home/ubuntu/tahoe100M_chunk_1")

# # artifact.path.download_to(store_path, print_progress=True, recursive=True, batch_size=64, )
# (artifact.path / "chunk_1.zarr").download_to(store_path, print_progress=True, recursive=True)

In [ ]:
adata = read_lazy(store_path)
var = pd.read_parquet("var.parquet")
print(var)
adata.var = var.reindex(adata.var.index)
print(adata)
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     adata = ad.concat([
#         read_lazy(store_path / chunk)
#         for chunk in tqdm(os.listdir(store_path))
#         if (store_path / chunk).is_dir()
#     ])


In [ ]:
adata.obs["y"] = adata.obs["cell_line"].astype("category").cat.codes.to_numpy().astype("i8")

In [ ]:
adata_train = adata[:800000]
adata_val = adata[800000:]

datamodule = ClassificationDataModule(
    adata_train=adata_train,
    adata_val=adata_val,
    label_column="y",
    train_dataloader_kwargs={
        "batch_size": 2048,
        "drop_last": True,
    },
    val_dataloader_kwargs={
        "batch_size": 2048,
        "drop_last": False,
    },
)

In [ ]:
linear = Linear(
    n_genes=adata.n_vars,
    n_covariates=adata.obs["y"].nunique(),
    learning_rate=1e-2,
)

In [ ]:
trainer = L.Trainer(
    max_epochs=3,
    log_every_n_steps=100,
    max_steps=3000,  # only fit a few steps for the sake of this tutorial
)

In [ ]:
trainer.fit(model=linear, datamodule=datamodule)

In [ ]:
# from LinearModuleAnalyzer import run_comprehensive_analysis

# analyzer, results = run_comprehensive_analysis(linear, adata, datamodule)

In [ ]:
# from UncertaintyEstimation import get_proper_uncertainty

# get_proper_uncertainty(linear, adata, datamodule)

In [ ]:
import os
os.chdir('/home/ubuntu/modlyn/mikaela')


In [ ]:
import importlib
import LinearModuleAnalyzer
importlib.reload(LinearModuleAnalyzer)

from LinearModuleAnalyzer import quick_analysis_with_scanpy_dotplot

analyzer, weight_adata, df = quick_analysis_with_scanpy_dotplot(linear, adata, datamodule)


In [ ]:
import importlib
import LinearModuleAnalyzer
importlib.reload(LinearModuleAnalyzer)
from LinearModuleAnalyzer import full_analysis

# Full analysis with all plots
results = full_analysis(linear, adata, datamodule)

In [ ]:
import importlib
import UncertaintyEstimation
importlib.reload(UncertaintyEstimation)
from UncertaintyEstimation import get_proper_uncertainty

results = get_proper_uncertainty(linear, adata, datamodule)


In [ ]:
# With interactive display
# results = get_proper_uncertainty(model, adata, datamodule, interactive=True)

# Access specific plots
# overview_fig = results['overview_fig']
# dashboard_fig = results['dashboard_fig']
# heatmap_fig = results['heatmap_fig']
# importance_fig = results['importance_fig']

In [ ]:
import importlib
import Figures
importlib.reload(Figures)
from Figures import create_publication_figures

nf, legends = create_publication_figures(linear, adata)

In [ ]:
# Example usage:
"""
# Quick exploration (5 minutes)
quick_analysis(model, adata)

# Full analysis (30 minutes)  
results = run_complete_analysis(model, adata, save_prefix="my_analysis")

# Access results
nature_figs = results['figures']['nature_figures']
bio_explorer = results['biology'] 
summary = results['summary_report']
paper_text = results['paper_text']
"""

In [ ]:
import importlib
import analysis_pipeline
importlib.reload(analysis_pipeline)
from analysis_pipeline import run_complete_analysis

# results = run_complete_analysis(linear, adata, save_prefix="my_analysis")

# Quick exploration (5 minutes)
gene_importance, top_genes = quick_analysis(linear, adata)




In [ ]:
# Full analysis (20-30 minutes)  
analyzer = run_complete_analysis(linear, adata)


In [ ]:
summary = analyzer.results
summary

In [ ]:
# Access results
# nature_figs = results['figures']['nature_figures']
# bio_explorer = results['biology'] 
# summary = results['summary_report']
# paper_text = results['paper_text']

In [ ]:
# answer_research_questions(analysis_results, linear)


In [ ]:
"""
# Create figure without data (uses mock results)
fig, caption = create_modlyn_figure()

# Create figure with actual model and data
fig, caption = create_modlyn_figure(model=your_model, adata=your_adata)
"""

In [ ]:
importlib.reload(OverviewFig)
import OverviewFig
from OverviewFig import create_modlyn_figure

fig, caption = create_modlyn_figure()

In [ ]:
# fig, caption = create_modlyn_figure(model=linear, adata=adata)


In [ ]:
import TahoeStory
importlib.reload(TahoeStory)
from TahoeStory import complete_tahoe_analysis
results = complete_tahoe_analysis(analyzer)
